<a href="https://colab.research.google.com/github/connect2srinu/Data-Science--Cheat-Sheet/blob/master/TEUsecaseV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


import gc
from datetime import datetime 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn import svm
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb

In [2]:
!pip install imblearn --upgrade
!pip install plotly --upgrade
!pip install chart-studio --upgrade

Requirement already up-to-date: imblearn in /usr/local/lib/python3.7/dist-packages (0.0)
Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)
Requirement already up-to-date: chart-studio in /usr/local/lib/python3.7/dist-packages (1.1.0)


In [3]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly as plotly
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import types
import pandas as pd
def __iter__(self): return 0

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



In [4]:
from google.colab import files
#files.upload()

In [5]:
data_df = pd.read_csv("predictive_maintainece_dataset.csv")

# New Section

In [6]:
data_df.describe()

,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9
count,124494.000000,1.244940e+05,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000
mean,0.000851,1.223881e+08,159.492706,9.940897,1.741120,14.222669,260172.657726,0.292528,0.292528,13.013848
std,0.029167,7.045933e+07,2179.677781,185.748131,22.908507,15.943028,99151.078547,7.436924,7.436924,275.661220
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,8.000000,0.000000,0.000000,0.000000
25%,0.000000,6.128476e+07,0.000000,0.000000,0.000000,8.000000,221452.000000,0.000000,0.000000,0.000000
50%,0.000000,1.227974e+08,0.000000,0.000000,0.000000,10.000000,249799.500000,0.000000,0.000000,0.000000
75%,0.000000,1.833096e+08,0.000000,0.000000,0.000000,12.000000,310266.000000,0.000000,0.000000,0.000000
max,1.000000,2.441405e+08,64968.000000,24929.000000,1666.000000,98.000000,689161.000000,832.000000,832.000000,70000.000000


In [7]:
data_df.head()


,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9
0,1/1/2015,S1F01085,0,215630672,55,0,52,6,407438,0,0,7
1,1/1/2015,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,1/1/2015,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,1/1/2015,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,1/1/2015,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


In [8]:
data_df.shape

(124494, 12)

In [9]:
data_df_device = pd.DataFrame(data_df.groupby(['device']).agg(['count']))
data_df_device.shape

(1169, 11)

In [10]:
data_df_date = pd.DataFrame(data_df.groupby(['date']).agg(['count']))
data_df_date.shape

(304, 11)

In [11]:
data_df.isnull().sum(axis = 0)

date       0
device     0
failure    0
metric1    0
metric2    0
metric3    0
metric4    0
metric5    0
metric6    0
metric7    0
metric8    0
metric9    0
dtype: int64

In [12]:
duplicate = data_df[data_df.duplicated(['device','date'])]
duplicate

,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9
101335,7/10/2015,S1F0R4Q8,0,192721392,0,0,0,8,213700,0,0,0


In [13]:
 data_df.duplicated(subset=['device','date'], keep='first')
#data_df[data_df.duplicated(subset=['device','date'], keep='first')]

0         False
1         False
2         False
3         False
4         False
          ...  
124489    False
124490    False
124491    False
124492    False
124493    False
Length: 124494, dtype: bool

In [14]:
data_df_class = pd.DataFrame(data_df.groupby(['failure'])['device'].agg('count'))
data_df_class

,device
failure,
0,124388
1,106


In [15]:
data_df['date'] = pd.to_datetime(data_df['date'])

In [16]:
data_df=data_df.sort_values(by=['device','date'], ascending=[True, True])
data_df['flipper'] = np.where((data_df.device != data_df.device.shift(1)), 1, 0)
data_df.head()

,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9,flipper
0,2015-01-01,S1F01085,0,215630672,55,0,52,6,407438,0,0,7,1
1163,2015-01-02,S1F01085,0,1650864,56,0,52,6,407438,0,0,7,0
2326,2015-01-03,S1F01085,0,124017368,56,0,52,6,407438,0,0,7,0
3489,2015-01-04,S1F01085,0,128073224,56,0,52,6,407439,0,0,7,0
4651,2015-01-05,S1F01085,0,97393448,56,0,52,6,408114,0,0,7,0


In [17]:
#define your feature window. This is the window by which we will aggregate our sensor values.
feature_window=21

In [18]:
dfx=data_df
#Select the first record of each machine
starter=dfx[dfx['flipper'] == 1]
starter=starter[['date','device']]
#rename date to START_DATE
starter=starter.rename(index=str, columns={"date": "START_DATE"})
#convert START_DATE to date
starter['START_DATE'] = pd.to_datetime(starter['START_DATE'])
#Merge START_DATE to the original data set
dfx=dfx.sort_values(by=['device', 'date'], ascending=[True, True])
starter=starter.sort_values(by=['device'], ascending=[True])
dfx =dfx.merge(starter, on=['device'], how='left')
# calculate the number of days since the beginning of each well. 
dfx['C'] = dfx['date'] - dfx['START_DATE']
dfx['TIME_SINCE_START'] = dfx['C'] / np.timedelta64(1, 'D')
dfx=dfx.drop(columns=['C'])
dfx['too_soon'] = np.where((dfx.TIME_SINCE_START < feature_window) , 1, 0)

In [19]:
dfx['metric1_mean'] = np.where((dfx.too_soon == 0),(dfx['metric1'].rolling(min_periods=1, window=feature_window).mean()) , dfx.metric1)
dfx['metric1_median'] = np.where((dfx.too_soon == 0),(dfx['metric1'].rolling(min_periods=1, window=feature_window).median()) , dfx.metric1)
dfx['metric1_max'] = np.where((dfx.too_soon == 0),(dfx['metric1'].rolling(min_periods=1, window=feature_window).max()) , dfx.metric1)
dfx['metric1_min'] = np.where((dfx.too_soon == 0),(dfx['metric1'].rolling(min_periods=1, window=feature_window).min()) , dfx.metric1)
dfx['metric2_mean'] = np.where((dfx.too_soon == 0),(dfx['metric2'].rolling(min_periods=1, window=feature_window).mean()) , dfx.metric2)
dfx['metric2_median'] = np.where((dfx.too_soon == 0),(dfx['metric2'].rolling(min_periods=1, window=feature_window).median()) , dfx.metric2)
dfx['metric2_max'] = np.where((dfx.too_soon == 0),(dfx['metric2'].rolling(min_periods=1, window=feature_window).max()) , dfx.metric2)
dfx['metric2_min'] = np.where((dfx.too_soon == 0),(dfx['metric2'].rolling(min_periods=1, window=feature_window).min()) , dfx.metric2)
dfx['metric3_mean'] = np.where((dfx.too_soon == 0),(dfx['metric3'].rolling(min_periods=1, window=feature_window).mean()) , dfx.metric3)
dfx['metric3_median'] = np.where((dfx.too_soon == 0),(dfx['metric3'].rolling(min_periods=1, window=feature_window).median()) , dfx.metric3)
dfx['metric3_max'] = np.where((dfx.too_soon == 0),(dfx['metric3'].rolling(min_periods=1, window=feature_window).max()) , dfx.metric3)
dfx['metric3_min'] = np.where((dfx.too_soon == 0),(dfx['metric3'].rolling(min_periods=1, window=feature_window).min()) , dfx.metric3)
dfx['metric4_mean'] = np.where((dfx.too_soon == 0),(dfx['metric4'].rolling(min_periods=1, window=feature_window).mean()) , dfx.metric4)
dfx['metric4_median'] = np.where((dfx.too_soon == 0),(dfx['metric4'].rolling(min_periods=1, window=feature_window).median()) , dfx.metric4)
dfx['metric4_max'] = np.where((dfx.too_soon == 0),(dfx['metric4'].rolling(min_periods=1, window=feature_window).max()) , dfx.metric4)
dfx['metric4_min'] = np.where((dfx.too_soon == 0),(dfx['metric4'].rolling(min_periods=1, window=feature_window).min()) , dfx.metric4)
dfx['metric5_mean'] = np.where((dfx.too_soon == 0),(dfx['metric5'].rolling(min_periods=1, window=feature_window).mean()) , dfx.metric5)
dfx['metric5_median'] = np.where((dfx.too_soon == 0),(dfx['metric5'].rolling(min_periods=1, window=feature_window).median()) , dfx.metric5)
dfx['metric5_max'] = np.where((dfx.too_soon == 0),(dfx['metric5'].rolling(min_periods=1, window=feature_window).max()) , dfx.metric5)
dfx['metric5_min'] = np.where((dfx.too_soon == 0),(dfx['metric5'].rolling(min_periods=1, window=feature_window).min()) , dfx.metric5)
dfx['metric6_mean'] = np.where((dfx.too_soon == 0),(dfx['metric6'].rolling(min_periods=1, window=feature_window).mean()) , dfx.metric6)
dfx['metric6_median'] = np.where((dfx.too_soon == 0),(dfx['metric6'].rolling(min_periods=1, window=feature_window).median()) , dfx.metric6)
dfx['metric6_max'] = np.where((dfx.too_soon == 0),(dfx['metric6'].rolling(min_periods=1, window=feature_window).max()) , dfx.metric6)
dfx['metric6_min'] = np.where((dfx.too_soon == 0),(dfx['metric6'].rolling(min_periods=1, window=feature_window).min()) , dfx.metric6)
dfx['metric7_mean'] = np.where((dfx.too_soon == 0),(dfx['metric7'].rolling(min_periods=1, window=feature_window).mean()) , dfx.metric7)
dfx['metric7_median'] = np.where((dfx.too_soon == 0),(dfx['metric7'].rolling(min_periods=1, window=feature_window).median()) , dfx.metric7)
dfx['metric7_max'] = np.where((dfx.too_soon == 0),(dfx['metric7'].rolling(min_periods=1, window=feature_window).max()) , dfx.metric7)
dfx['metric7_min'] = np.where((dfx.too_soon == 0),(dfx['metric7'].rolling(min_periods=1, window=feature_window).min()) , dfx.metric7)
dfx['metric8_mean'] = np.where((dfx.too_soon == 0),(dfx['metric8'].rolling(min_periods=1, window=feature_window).mean()) , dfx.metric8)
dfx['metric8_median'] = np.where((dfx.too_soon == 0),(dfx['metric8'].rolling(min_periods=1, window=feature_window).median()) , dfx.metric8)
dfx['metric8_max'] = np.where((dfx.too_soon == 0),(dfx['metric8'].rolling(min_periods=1, window=feature_window).max()) , dfx.metric8)
dfx['metric8_min'] = np.where((dfx.too_soon == 0),(dfx['metric8'].rolling(min_periods=1, window=feature_window).min()) , dfx.metric8)
dfx['metric9_mean'] = np.where((dfx.too_soon == 0),(dfx['metric9'].rolling(min_periods=1, window=feature_window).mean()) , dfx.metric9)
dfx['metric9_median'] = np.where((dfx.too_soon == 0),(dfx['metric9'].rolling(min_periods=1, window=feature_window).median()) , dfx.metric9)
dfx['metric9_max'] = np.where((dfx.too_soon == 0),(dfx['metric9'].rolling(min_periods=1, window=feature_window).max()) , dfx.metric9)
dfx['metric9_min'] = np.where((dfx.too_soon == 0),(dfx['metric9'].rolling(min_periods=1, window=feature_window).min()) , dfx.metric9)

In [20]:
dfx['metric1_chg'] = np.where((dfx.metric1_mean == 0),0 , dfx.metric1/dfx.metric1_mean)
dfx['metric2_chg'] = np.where((dfx.metric2_mean == 0),0 , dfx.metric2/dfx.metric2_mean)
dfx['metric3_chg'] = np.where((dfx.metric3_mean==0),0 , dfx.metric3/dfx.metric3_mean)
dfx['metric4_chg'] = np.where((dfx.metric4_mean == 0),0 , dfx.metric4/dfx.metric4_mean)
dfx['metric5_chg'] = np.where((dfx.metric5_mean == 0),0 , dfx.metric5/dfx.metric5_mean)
dfx['metric6_chg'] = np.where((dfx.metric6_mean == 0),0 , dfx.metric6/dfx.metric6_mean)
dfx['metric7_chg'] = np.where((dfx.metric7_mean == 0),0 , dfx.metric7/dfx.metric7_mean)
dfx['metric8_chg'] = np.where((dfx.metric8_mean == 0),0 , dfx.metric8/dfx.metric8_mean)
dfx['metric9_chg'] = np.where((dfx.metric9_mean == 0),0 , dfx.metric9/dfx.metric9_mean)
#copy the data set to the original name
data_df=dfx

In [21]:
data_df.head()

,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9,flipper,START_DATE,TIME_SINCE_START,too_soon,metric1_mean,metric1_median,metric1_max,metric1_min,metric2_mean,metric2_median,metric2_max,metric2_min,metric3_mean,metric3_median,metric3_max,metric3_min,metric4_mean,metric4_median,metric4_max,metric4_min,metric5_mean,metric5_median,metric5_max,metric5_min,metric6_mean,metric6_median,metric6_max,metric6_min,metric7_mean,metric7_median,metric7_max,metric7_min,metric8_mean,metric8_median,metric8_max,metric8_min,metric9_mean,metric9_median,metric9_max,metric9_min,metric1_chg,metric2_chg,metric3_chg,metric4_chg,metric5_chg,metric6_chg,metric7_chg,metric8_chg,metric9_chg
0,2015-01-01,S1F01085,0,215630672,55,0,52,6,407438,0,0,7,1,2015-01-01,0.0,1,215630672.0,215630672.0,215630672.0,215630672.0,55.0,55.0,55.0,55.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
1,2015-01-02,S1F01085,0,1650864,56,0,52,6,407438,0,0,7,0,2015-01-01,1.0,1,1650864.0,1650864.0,1650864.0,1650864.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
2,2015-01-03,S1F01085,0,124017368,56,0,52,6,407438,0,0,7,0,2015-01-01,2.0,1,124017368.0,124017368.0,124017368.0,124017368.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
3,2015-01-04,S1F01085,0,128073224,56,0,52,6,407439,0,0,7,0,2015-01-01,3.0,1,128073224.0,128073224.0,128073224.0,128073224.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407439.0,407439.0,407439.0,407439.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
4,2015-01-05,S1F01085,0,97393448,56,0,52,6,408114,0,0,7,0,2015-01-01,4.0,1,97393448.0,97393448.0,97393448.0,97393448.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,408114.0,408114.0,408114.0,408114.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0


In [22]:
data_df.head()

,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9,flipper,START_DATE,TIME_SINCE_START,too_soon,metric1_mean,metric1_median,metric1_max,metric1_min,metric2_mean,metric2_median,metric2_max,metric2_min,metric3_mean,metric3_median,metric3_max,metric3_min,metric4_mean,metric4_median,metric4_max,metric4_min,metric5_mean,metric5_median,metric5_max,metric5_min,metric6_mean,metric6_median,metric6_max,metric6_min,metric7_mean,metric7_median,metric7_max,metric7_min,metric8_mean,metric8_median,metric8_max,metric8_min,metric9_mean,metric9_median,metric9_max,metric9_min,metric1_chg,metric2_chg,metric3_chg,metric4_chg,metric5_chg,metric6_chg,metric7_chg,metric8_chg,metric9_chg
0,2015-01-01,S1F01085,0,215630672,55,0,52,6,407438,0,0,7,1,2015-01-01,0.0,1,215630672.0,215630672.0,215630672.0,215630672.0,55.0,55.0,55.0,55.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
1,2015-01-02,S1F01085,0,1650864,56,0,52,6,407438,0,0,7,0,2015-01-01,1.0,1,1650864.0,1650864.0,1650864.0,1650864.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
2,2015-01-03,S1F01085,0,124017368,56,0,52,6,407438,0,0,7,0,2015-01-01,2.0,1,124017368.0,124017368.0,124017368.0,124017368.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
3,2015-01-04,S1F01085,0,128073224,56,0,52,6,407439,0,0,7,0,2015-01-01,3.0,1,128073224.0,128073224.0,128073224.0,128073224.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407439.0,407439.0,407439.0,407439.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
4,2015-01-05,S1F01085,0,97393448,56,0,52,6,408114,0,0,7,0,2015-01-01,4.0,1,97393448.0,97393448.0,97393448.0,97393448.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,408114.0,408114.0,408114.0,408114.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0


In [23]:
target_window=28


In [24]:
data_df=data_df.sort_values(by=['device', 'date'], ascending=[True, True])
data_df.reset_index(level=0, inplace=True)

In [25]:
data_df.head()

,index,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9,flipper,START_DATE,TIME_SINCE_START,too_soon,metric1_mean,metric1_median,metric1_max,metric1_min,metric2_mean,metric2_median,metric2_max,metric2_min,metric3_mean,metric3_median,metric3_max,metric3_min,metric4_mean,metric4_median,metric4_max,metric4_min,metric5_mean,metric5_median,metric5_max,metric5_min,metric6_mean,metric6_median,metric6_max,metric6_min,metric7_mean,metric7_median,metric7_max,metric7_min,metric8_mean,metric8_median,metric8_max,metric8_min,metric9_mean,metric9_median,metric9_max,metric9_min,metric1_chg,metric2_chg,metric3_chg,metric4_chg,metric5_chg,metric6_chg,metric7_chg,metric8_chg,metric9_chg
0,0,2015-01-01,S1F01085,0,215630672,55,0,52,6,407438,0,0,7,1,2015-01-01,0.0,1,215630672.0,215630672.0,215630672.0,215630672.0,55.0,55.0,55.0,55.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
1,1,2015-01-02,S1F01085,0,1650864,56,0,52,6,407438,0,0,7,0,2015-01-01,1.0,1,1650864.0,1650864.0,1650864.0,1650864.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
2,2,2015-01-03,S1F01085,0,124017368,56,0,52,6,407438,0,0,7,0,2015-01-01,2.0,1,124017368.0,124017368.0,124017368.0,124017368.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
3,3,2015-01-04,S1F01085,0,128073224,56,0,52,6,407439,0,0,7,0,2015-01-01,3.0,1,128073224.0,128073224.0,128073224.0,128073224.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407439.0,407439.0,407439.0,407439.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
4,4,2015-01-05,S1F01085,0,97393448,56,0,52,6,408114,0,0,7,0,2015-01-01,4.0,1,97393448.0,97393448.0,97393448.0,97393448.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,408114.0,408114.0,408114.0,408114.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0


In [26]:
df_failure_thingy=data_df[data_df['failure'] == 1]
df_failure_thingy=df_failure_thingy[['date','device']]
df_failure_thingy=df_failure_thingy.rename(index=str, columns={"date": "FAILURE_DATE"})
data_df=data_df.sort_values(by=['device'], ascending=[True])
df_failure_thingy=df_failure_thingy.sort_values(by=['device'], ascending=[True])

In [27]:
data_df =data_df.merge(df_failure_thingy, on=['device'], how='left')

In [28]:
data_df.head()

,index,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9,flipper,START_DATE,TIME_SINCE_START,too_soon,metric1_mean,metric1_median,metric1_max,metric1_min,metric2_mean,metric2_median,metric2_max,metric2_min,metric3_mean,metric3_median,metric3_max,metric3_min,metric4_mean,metric4_median,metric4_max,metric4_min,metric5_mean,metric5_median,metric5_max,metric5_min,metric6_mean,metric6_median,metric6_max,metric6_min,metric7_mean,metric7_median,metric7_max,metric7_min,metric8_mean,metric8_median,metric8_max,metric8_min,metric9_mean,metric9_median,metric9_max,metric9_min,metric1_chg,metric2_chg,metric3_chg,metric4_chg,metric5_chg,metric6_chg,metric7_chg,metric8_chg,metric9_chg,FAILURE_DATE
0,0,2015-01-01,S1F01085,0,215630672,55,0,52,6,407438,0,0,7,1,2015-01-01,0.0,1,215630672.0,215630672.0,215630672.0,215630672.0,55.0,55.0,55.0,55.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaT
1,1,2015-01-02,S1F01085,0,1650864,56,0,52,6,407438,0,0,7,0,2015-01-01,1.0,1,1650864.0,1650864.0,1650864.0,1650864.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaT
2,2,2015-01-03,S1F01085,0,124017368,56,0,52,6,407438,0,0,7,0,2015-01-01,2.0,1,124017368.0,124017368.0,124017368.0,124017368.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaT
3,3,2015-01-04,S1F01085,0,128073224,56,0,52,6,407439,0,0,7,0,2015-01-01,3.0,1,128073224.0,128073224.0,128073224.0,128073224.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407439.0,407439.0,407439.0,407439.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaT
4,4,2015-01-05,S1F01085,0,97393448,56,0,52,6,408114,0,0,7,0,2015-01-01,4.0,1,97393448.0,97393448.0,97393448.0,97393448.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,408114.0,408114.0,408114.0,408114.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaT


In [29]:
data_df=data_df.sort_values(by=['device','date'], ascending=[True, True])
data_df['FAILURE_DATE'] = pd.to_datetime(data_df['FAILURE_DATE'])
data_df['date'] = pd.to_datetime(data_df['date'])
data_df['C'] = data_df['FAILURE_DATE'] - data_df['date']
data_df['TIME_TO_FAILURE'] = data_df['C'] / np.timedelta64(1, 'D')

In [30]:
data_df.head()
pd_data=data_df

In [31]:
pd_data=pd_data.drop(columns=['index'])
pd_data=pd_data.sort_values(by=['device', 'date'], ascending=[True, True])
pd_data.reset_index(inplace=True)

In [32]:
pd_data['FAILURE_TARGET'] = np.where(((pd_data.TIME_TO_FAILURE < target_window) & ((pd_data.TIME_TO_FAILURE>=0))), 1, 0)
tips_summed = pd_data.groupby(['FAILURE_TARGET'])['metric1'].count()
tips_summed

FAILURE_TARGET
0    121912
1      2582
Name: metric1, dtype: int64

In [33]:
pd_data['FAILURE_TARGET'].mean()

0.020739955339213134

In [34]:
#Get a Unique List of All IDs
aa=pd_data
pd_id=aa.drop_duplicates(subset='device')
pd_id=pd_id[['device']]
pd_id.shape

(1169, 1)

In [35]:
np.random.seed(42)
pd_id['wookie'] = (np.random.randint(0, 10000, pd_id.shape[0]))/10000
pd_id=pd_id[['device', 'wookie']]

In [36]:
pd_id['MODELING_GROUP'] = np.where(((pd_id.wookie <= 0.80)), 'TRAINING', np.where(((pd_id.wookie <= 0.90)), 'VALIDATION', 'TESTING'))

In [37]:
tips_summed = pd_id.groupby(['MODELING_GROUP'])['wookie'].count()
tips_summed

MODELING_GROUP
TESTING        99
TRAINING      966
VALIDATION    104
Name: wookie, dtype: int64

In [38]:
pd_data=pd_data.sort_values(by=['device'], ascending=[True])
pd_id=pd_id.sort_values(by=['device'], ascending=[True])
pd_data =pd_data.merge(pd_id, on=['device'], how='inner')

In [39]:
tips_summed = pd_data.groupby(['MODELING_GROUP'])['wookie'].count()
tips_summed

MODELING_GROUP
TESTING        11738
TRAINING      101231
VALIDATION     11525
Name: wookie, dtype: int64

In [40]:
tips_summed = pd_data.groupby(['MODELING_GROUP'])['FAILURE_TARGET'].sum()
tips_summed

MODELING_GROUP
TESTING        181
TRAINING      2188
VALIDATION     213
Name: FAILURE_TARGET, dtype: int64

In [41]:
df_training=pd_data[pd_data['MODELING_GROUP'] == 'TRAINING']
df_training=df_training.drop(columns=['MODELING_GROUP','C','wookie','TIME_TO_FAILURE','flipper','START_DATE'])
df_training.shape

(101231, 62)

In [42]:
df_train_test=pd_data[pd_data['MODELING_GROUP'] != 'VALIDATION']
df_train_test=df_train_test.drop(columns=['wookie','TIME_TO_FAILURE','flipper','START_DATE'])
df_train_test.shape

(112969, 64)

In [43]:
df_total=pd_data.drop(columns=['C','wookie','TIME_TO_FAILURE','flipper','START_DATE'])

In [44]:
df_total.columns


Index(['index', 'date', 'device', 'failure', 'metric1', 'metric2', 'metric3',
       'metric4', 'metric5', 'metric6', 'metric7', 'metric8', 'metric9',
       'TIME_SINCE_START', 'too_soon', 'metric1_mean', 'metric1_median',
       'metric1_max', 'metric1_min', 'metric2_mean', 'metric2_median',
       'metric2_max', 'metric2_min', 'metric3_mean', 'metric3_median',
       'metric3_max', 'metric3_min', 'metric4_mean', 'metric4_median',
       'metric4_max', 'metric4_min', 'metric5_mean', 'metric5_median',
       'metric5_max', 'metric5_min', 'metric6_mean', 'metric6_median',
       'metric6_max', 'metric6_min', 'metric7_mean', 'metric7_median',
       'metric7_max', 'metric7_min', 'metric8_mean', 'metric8_median',
       'metric8_max', 'metric8_min', 'metric9_mean', 'metric9_median',
       'metric9_max', 'metric9_min', 'metric1_chg', 'metric2_chg',
       'metric3_chg', 'metric4_chg', 'metric5_chg', 'metric6_chg',
       'metric7_chg', 'metric8_chg', 'metric9_chg', 'FAILURE_DATE',
      

In [45]:
# training_features=df_training[['index', 'date', 'device', 'failure', 'metric1', 'metric2', 'metric3',
#        'metric4', 'metric5', 'metric6', 'metric7', 'metric8', 'metric9',
#        'metric1_mean', 'metric1_median',
#        'metric1_max', 'metric1_min', 'metric2_mean', 'metric2_median',
#        'metric2_max', 'metric2_min', 'metric3_mean', 'metric3_median',
#        'metric3_max', 'metric3_min', 'metric4_mean', 'metric4_median',
#        'metric4_max', 'metric4_min', 'metric5_mean', 'metric5_median',
#        'metric5_max', 'metric5_min', 'metric6_mean', 'metric6_median',
#        'metric6_max', 'metric6_min', 'metric7_mean', 'metric7_median',
#        'metric7_max', 'metric7_min', 'metric8_mean', 'metric8_median',
#        'metric8_max', 'metric8_min', 'metric9_mean', 'metric9_median',
#        'metric9_max', 'metric9_min', 'metric1_chg', 'metric2_chg',
#        'metric3_chg', 'metric4_chg', 'metric5_chg', 'metric6_chg',
#        'metric7_chg', 'metric8_chg', 'metric9_chg']]
# training_target=df_training[['FAILURE_TARGET']]

training_features=df_training[[  'metric1', 'metric2', 'metric3',
       'metric4', 'metric5', 'metric6', 'metric7', 'metric8', 'metric9',
       'metric1_mean', 'metric1_median',
       'metric1_max', 'metric1_min', 'metric2_mean', 'metric2_median',
       'metric2_max', 'metric2_min', 'metric3_mean', 'metric3_median',
       'metric3_max', 'metric3_min', 'metric4_mean', 'metric4_median',
       'metric4_max', 'metric4_min', 'metric5_mean', 'metric5_median',
       'metric5_max', 'metric5_min', 'metric6_mean', 'metric6_median',
       'metric6_max', 'metric6_min', 'metric7_mean', 'metric7_median',
       'metric7_max', 'metric7_min', 'metric8_mean', 'metric8_median',
       'metric8_max', 'metric8_min', 'metric9_mean', 'metric9_median',
       'metric9_max', 'metric9_min', 'metric1_chg', 'metric2_chg',
       'metric3_chg', 'metric4_chg', 'metric5_chg', 'metric6_chg',
       'metric7_chg', 'metric8_chg', 'metric9_chg']]
training_target=df_training[['FAILURE_TARGET']]


In [46]:
training_target.value_counts()

FAILURE_TARGET
0                 99043
1                  2188
dtype: int64

In [47]:
df_testing=pd_data[pd_data['MODELING_GROUP'] == 'TESTING']
df_testing=df_testing.drop(columns=['MODELING_GROUP','C','wookie','TIME_TO_FAILURE','flipper','START_DATE'])
df_testing.shape
testing_features=df_testing[[  'metric1', 'metric2', 'metric3',
       'metric4', 'metric5', 'metric6', 'metric7', 'metric8', 'metric9',
       'metric1_mean', 'metric1_median',
       'metric1_max', 'metric1_min', 'metric2_mean', 'metric2_median',
       'metric2_max', 'metric2_min', 'metric3_mean', 'metric3_median',
       'metric3_max', 'metric3_min', 'metric4_mean', 'metric4_median',
       'metric4_max', 'metric4_min', 'metric5_mean', 'metric5_median',
       'metric5_max', 'metric5_min', 'metric6_mean', 'metric6_median',
       'metric6_max', 'metric6_min', 'metric7_mean', 'metric7_median',
       'metric7_max', 'metric7_min', 'metric8_mean', 'metric8_median',
       'metric8_max', 'metric8_min', 'metric9_mean', 'metric9_median',
       'metric9_max', 'metric9_min', 'metric1_chg', 'metric2_chg',
       'metric3_chg', 'metric4_chg', 'metric5_chg', 'metric6_chg',
       'metric7_chg', 'metric8_chg', 'metric9_chg']]
testing_target=df_testing[['FAILURE_TARGET']]

In [48]:
##Step1
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
print(len(training_features.columns))
n_inputs=55

54


In [49]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import os

In [50]:
# def seq2seq_window_dataset(series, window_size, batch_size=32,
#                            shuffle_buffer=1000):
#     series = tf.expand_dims(series, axis=-1)
#     ds = tf.data.Dataset.from_tensor_slices(series)
#     ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
#     ds = ds.flat_map(lambda w: w.batch(window_size + 1))
#     ds = ds.shuffle(shuffle_buffer)
#     ds = ds.map(lambda w: (w[:-1], w[1:]))
#     return ds.batch(batch_size).prefetch(1)

# def model_cnn_full():
#     model = keras.models.Sequential()
#     model.add(keras.layers.InputLayer(input_shape=[None, 1]))
#     for dilation_rate in (1, 2, 4, 8, 16, 32):
#         model.add(
#             keras.layers.Conv1D(filters=32,
#                                 kernel_size=2,
#                                 strides=1,
#                                 dilation_rate=dilation_rate,
#                                 padding="causal",
#                                 activation="relu")
#         )
#         return model

# def model_forecast(model, series, window_size):
#     ds = tf.data.Dataset.from_tensor_slices(series)
#     ds = ds.window(window_size, shift=1, drop_remainder=True)
#     ds = ds.flat_map(lambda w: w.batch(window_size))
#     ds = ds.batch(32).prefetch(1)
#     forecast = model.predict(ds)
#     return forecast



# keras.backend.clear_session()
# tf.random.set_seed(42)
# np.random.seed(42)

# # Scale each feature to 0-1
# scaler = MinMaxScaler(feature_range = (0, 1))
# # Repeat with the scaler
# scaler.fit(training_features)
# training_features = scaler.transform(training_features)
# testing_features = scaler.transform(testing_features)

# print('Training data shape: ', training_features.shape)
# print('Testing data shape: ', testing_features.shape)

# window_size = 64
# train_set = seq2seq_window_dataset(training_features, window_size,
#                                    batch_size=128)
# valid_set = seq2seq_window_dataset(testing_features, window_size,
#                                    batch_size=128)



# class F1Score(tf.keras.metrics.Metric):
#   def __init__(self, name='F1Score', **kwargs):
#     super(F1Score, self).__init__(name=name, **kwargs)
#     self.f1score = self.add_weight(name='F1Score', initializer='zeros')
#     self.count = self.add_weight(name='F1ScoreCount', initializer='zeros')

#   def update_state(self, y_true, y_pred, sample_weight=None):
#     y_true = tf.cast(y_true, tf.bool)
#     y_pred = tf.cast(y_pred, tf.bool)

#     true_positives = tf.logical_and(tf.equal(y_true, True), tf.equal(y_pred, True))
#     true_positives = tf.cast(true_positives, self.dtype)
#     count_true_positives = tf.reduce_sum(true_positives)

#     possible_positives = tf.cast(y_true, self.dtype)
#     count_possible_positives = tf.reduce_sum(possible_positives)

#     predicted_positives = tf.cast(y_pred, self.dtype)
#     count_predicted_positives = tf.reduce_sum(predicted_positives)

#     precision = count_true_positives / (count_predicted_positives + tf.keras.backend.epsilon())
#     recall = count_true_positives / (count_possible_positives + tf.keras.backend.epsilon())
#     f1_cal = 2*(precision*recall)/(precision + recall + tf.keras.backend.epsilon())

#     self.count.assign_add(1)
#     a = 1.0 / self.count
#     b = 1.0 - a
#     self.f1score.assign(a*f1_cal+b*self.f1score)

#   def result(self):
#     return self.f1score


In [51]:
# testing_features=df_testing[[  'metric1', 'metric2', 'metric3',
#        'metric4', 'metric5', 'metric6', 'metric7', 'metric8', 'metric9',
#        'metric1_mean', 'metric1_median',
#        'metric1_max', 'metric1_min', 'metric2_mean', 'metric2_median',
#        'metric2_max', 'metric2_min', 'metric3_mean', 'metric3_median',
#        'metric3_max', 'metric3_min', 'metric4_mean', 'metric4_median',
#        'metric4_max', 'metric4_min', 'metric5_mean', 'metric5_median',
#        'metric5_max', 'metric5_min', 'metric6_mean', 'metric6_median',
#        'metric6_max', 'metric6_min', 'metric7_mean', 'metric7_median',
#        'metric7_max', 'metric7_min', 'metric8_mean', 'metric8_median',
#        'metric8_max', 'metric8_min', 'metric9_mean', 'metric9_median',
#        'metric9_max', 'metric9_min', 'metric1_chg', 'metric2_chg',
#        'metric3_chg', 'metric4_chg', 'metric5_chg', 'metric6_chg',
#        'metric7_chg', 'metric8_chg', 'metric9_chg']]
# testing_target=df_testing[['FAILURE_TARGET']]
features_col_name=['metric1', 'metric2', 'metric3',
       'metric4', 'metric5', 'metric6', 'metric7', 'metric8', 'metric9',
       'metric1_mean', 'metric1_median',
       'metric1_max', 'metric1_min', 'metric2_mean', 'metric2_median',
       'metric2_max', 'metric2_min', 'metric3_mean', 'metric3_median',
       'metric3_max', 'metric3_min', 'metric4_mean', 'metric4_median',
       'metric4_max', 'metric4_min', 'metric5_mean', 'metric5_median',
       'metric5_max', 'metric5_min', 'metric6_mean', 'metric6_median',
       'metric6_max', 'metric6_min', 'metric7_mean', 'metric7_median',
       'metric7_max', 'metric7_min', 'metric8_mean', 'metric8_median',
       'metric8_max', 'metric8_min', 'metric9_mean', 'metric9_median',
       'metric9_max', 'metric9_min', 'metric1_chg', 'metric2_chg',
       'metric3_chg', 'metric4_chg', 'metric5_chg', 'metric6_chg',
       'metric7_chg', 'metric8_chg', 'metric9_chg']
target_col_name='FAILURE_TARGET'


In [52]:
sc=MinMaxScaler()
df_training[features_col_name]=sc.fit_transform(df_training[features_col_name])
df_testing[features_col_name]=sc.transform(df_testing[features_col_name])
print(df_training.shape)
print(df_testing.shape)

(101231, 62)
(11738, 62)


In [66]:
def gen_sequence(id_df, seq_length, seq_cols):
    df_zeros=pd.DataFrame(np.zeros((seq_length-1,id_df.shape[1])),columns=id_df.columns)
    id_df=df_zeros.append(id_df,ignore_index=True)
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    lstm_array=[]
    print(data_array['device'])
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        lstm_array.append(data_array[start:stop, :])
    return np.array(lstm_array)

# function to generate labels
def gen_label(id_df, seq_length, seq_cols,label):
    df_zeros=pd.DataFrame(np.zeros((seq_length-1,id_df.shape[1])),columns=id_df.columns)
    id_df=df_zeros.append(id_df,ignore_index=True)
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    y_label=[]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        y_label.append(id_df[label][stop])
    return np.array(y_label)

In [61]:
len(df_training['device'].unique())

966

In [67]:
# timestamp or window size
seq_length=50
seq_cols=features_col_name
X_train=np.concatenate(list(list(gen_sequence(df_training[df_training['device']==device], seq_length, seq_cols)) for device in df_training['device'].unique()))
print(X_train.shape)
# generate y_train
y_train=np.concatenate(list(list(gen_label(df_training[df_training['device']==device], 50, seq_cols,'FAILURE_TARGET')) for device in df_training['device'].unique()))
print(y_train.shape)

ValueError: ignored

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for dilation_rate in (1, 2, 4, 8, 16, 32):
    model.add(
      keras.layers.Conv1D(filters=32,
                          kernel_size=2,
                          strides=1,
                          dilation_rate=dilation_rate,
                          padding="causal",
                          activation="relu")
    )
#model.add(keras.layers.Conv1D(filters=1, kernel_size=1))
model.add(keras.layers.Dense(1, activation='sigmoid'))

lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 30))
optimizer = keras.optimizers.Adam(lr=1e-4)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy',  F1Score()])
model.summary()
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

  # define loss and optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',  F1Score()])
model.summary()
model.fit(training_features,training_target,epochs=10, verbose=1,batch_size=512)

In [75]:
# prepro = MinMaxScaler()
# prepro.fit(training_features)
# prepro_test = MinMaxScaler()
# prepro_test.fit(testing_features)
training_features=df_training[[ 'metric1', 'metric2', 'metric3',
       'metric4', 'metric5', 'metric6', 'metric7', 'metric8', 'metric9']]
training_target=df_training[['FAILURE_TARGET']]
testing_features=df_testing[[  'metric1', 'metric2', 'metric3',
       'metric4', 'metric5', 'metric6', 'metric7', 'metric8', 'metric9']]
testing_target=df_testing[['FAILURE_TARGET']]

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))
# Repeat with the scaler
scaler.fit(training_features)
training_features = scaler.transform(training_features)
testing_features = scaler.transform(testing_features)

print('Training data shape: ', training_features.shape)
print('Testing data shape: ', testing_features.shape)

Training data shape:  (101231, 9)
Testing data shape:  (11738, 9)


In [70]:
print(training_features.shape)
training_features_df = pd.DataFrame(training_features)
training_features_df.describe()

(101231, 9)


,0,1,2,3,4,5,6,7,8
count,101231.000000,101231.000000,101231.000000,101231.000000,101231.000000,101231.000000,101231.000000,101231.000000,101231.000000
mean,0.501020,0.002599,0.000339,0.001218,0.142590,0.377937,0.000440,0.000440,0.000775
std,0.288779,0.034319,0.007836,0.015231,0.173484,0.144497,0.009878,0.009878,0.011341
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250530,0.000000,0.000000,0.000000,0.074468,0.321222,0.000000,0.000000,0.000000
50%,0.502497,0.000000,0.000000,0.000000,0.095745,0.362522,0.000000,0.000000,0.000000
75%,0.750920,0.000000,0.000000,0.000000,0.117021,0.450142,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [71]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import Accuracy, BinaryAccuracy
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf

class F1Score(tf.keras.metrics.Metric):
  def __init__(self, name='F1Score', **kwargs):
    super(F1Score, self).__init__(name=name, **kwargs)
    self.f1score = self.add_weight(name='F1Score', initializer='zeros')
    self.count = self.add_weight(name='F1ScoreCount', initializer='zeros')

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_true = tf.cast(y_true, tf.bool)
    y_pred = tf.cast(y_pred, tf.bool)

    true_positives = tf.logical_and(tf.equal(y_true, True), tf.equal(y_pred, True))
    true_positives = tf.cast(true_positives, self.dtype)
    count_true_positives = tf.reduce_sum(true_positives)

    possible_positives = tf.cast(y_true, self.dtype)
    count_possible_positives = tf.reduce_sum(possible_positives)

    predicted_positives = tf.cast(y_pred, self.dtype)
    count_predicted_positives = tf.reduce_sum(predicted_positives)

    precision = count_true_positives / (count_predicted_positives + tf.keras.backend.epsilon())
    recall = count_true_positives / (count_possible_positives + tf.keras.backend.epsilon())
    f1_cal = 2*(precision*recall)/(precision + recall + tf.keras.backend.epsilon())

    self.count.assign_add(1)
    a = 1.0 / self.count
    b = 1.0 - a
    self.f1score.assign(a*f1_cal+b*self.f1score)

  def result(self):
    return self.f1score

In [61]:
training_features.shape[1]

54

In [80]:
n_inputs=training_features.shape[1]
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_dim=n_inputs),
  tf.keras.layers.Dense(128, activation='relu'),  
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])
  # define loss and optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',  F1Score()])
model.summary()
model.fit(training_features,training_target,epochs=10, verbose=1,batch_size=512)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 9)                 0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               1280      
_________________________________________________________________
dense_20 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 257       
Total params: 34,561
Trainable params: 34,561
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
198/198 [==============================] - 2s 6ms/step - loss: 0.2268 - accuracy: 0.9782 - F1Score: 0.0418
Epoch 2/10
198/198 [==============================] - 1s 6ms/step - loss: 0.0995 - accuracy: 0.9783 - F1Score: 0.0422
Epoch 3/10
1

In [79]:
from sklearn.metrics import roc_auc_score
y_pred1=model.predict(testing_features)

print(roc_auc_score(testing_target,y_pred1))

0.4775477491577896


In [101]:
# define model
model = Sequential()
# define first hidden layer and visible layer
model.add(Dense(128, input_dim=n_inputs, activation='relu'))
model.add(Dense(256,  activation='relu'))
# define output layer
model.add(Dense(1, activation='sigmoid'))
# define loss and optimizer
model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()
model.fit(training_features,training_target,epochs=10,batch_size=256)


Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 128)               7168      
_________________________________________________________________
dense_35 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 257       
Total params: 40,449
Trainable params: 40,449
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
396/396 [==============================] - 2s 4ms/step - loss: 0.1567
Epoch 2/10
396/396 [==============================] - 2s 4ms/step - loss: 0.0827
Epoch 3/10
396/396 [==============================] - 2s 4ms/step - loss: 0.0827
Epoch 4/10
396/396 [==============================] - 2s 4ms/step - loss: 0.0815
Epoch 5/10
396/396 [==============================] - 2

In [102]:
from sklearn.metrics import roc_auc_score
#testing_target = pd.DataFrame(testing_target)
y_pred1=model.predict(testing_features)
print(y_pred1.shape)
print(testing_target.shape)
print(roc_auc_score(testing_target,y_pred1))

# Evaluating confusion matric
res = tf.math.confusion_matrix(testing_target,y_pred1,num_classes=2, dtype=tf.dtypes.int32,)
  
# Printing the result
print('Confusion_matrix: ',res)

(11738, 1)
(11738, 1)
0.5


InvalidArgumentError: ignored

In [53]:
weights_assigned={0:1,1:45}

In [87]:
# define model
model = Sequential()
# define first hidden layer and visible layer
model.add(Dense(100, input_dim=n_inputs, activation='relu', kernel_initializer='he_uniform'))
# define output layer
model.add(Dense(1, activation='sigmoid'))
# define loss and optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',  F1Score()])
model.fit(training_features,training_target,class_weight=weights_assigned,epochs=10, verbose=1,batch_size=512)

Epoch 1/10
198/198 [==============================] - 1s 3ms/step - loss: 1.3435 - accuracy: 0.4786 - F1Score: 0.0441
Epoch 2/10
198/198 [==============================] - 1s 3ms/step - loss: 1.1908 - accuracy: 0.8020 - F1Score: 0.0416
Epoch 3/10
198/198 [==============================] - 1s 3ms/step - loss: 1.1368 - accuracy: 0.8596 - F1Score: 0.0430
Epoch 4/10
198/198 [==============================] - 1s 3ms/step - loss: 1.0892 - accuracy: 0.8799 - F1Score: 0.0420
Epoch 5/10
198/198 [==============================] - 1s 3ms/step - loss: 1.0691 - accuracy: 0.8720 - F1Score: 0.0423
Epoch 6/10
198/198 [==============================] - 1s 3ms/step - loss: 1.0685 - accuracy: 0.8752 - F1Score: 0.0433
Epoch 7/10
198/198 [==============================] - 1s 3ms/step - loss: 1.0516 - accuracy: 0.8764 - F1Score: 0.0429
Epoch 8/10
198/198 [==============================] - 1s 3ms/step - loss: 1.0416 - accuracy: 0.8741 - F1Score: 0.0428
Epoch 9/10
198/198 [==============================] - 1s

In [88]:
from sklearn.metrics import roc_auc_score
y_pred1=model.predict(testing_features)

print(roc_auc_score(testing_target,y_pred1))

0.7867141341713926


In [58]:
df_training.head()

,index,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9,TIME_SINCE_START,too_soon,metric1_mean,metric1_median,metric1_max,metric1_min,metric2_mean,metric2_median,metric2_max,metric2_min,metric3_mean,metric3_median,metric3_max,metric3_min,metric4_mean,metric4_median,metric4_max,metric4_min,metric5_mean,metric5_median,metric5_max,metric5_min,metric6_mean,metric6_median,metric6_max,metric6_min,metric7_mean,metric7_median,metric7_max,metric7_min,metric8_mean,metric8_median,metric8_max,metric8_min,metric9_mean,metric9_median,metric9_max,metric9_min,metric1_chg,metric2_chg,metric3_chg,metric4_chg,metric5_chg,metric6_chg,metric7_chg,metric8_chg,metric9_chg,FAILURE_DATE,FAILURE_TARGET
0,0,2015-01-01,S1F01085,0,215630672,55,0,52,6,407438,0,0,7,0.0,1,215630672.0,215630672.0,215630672.0,215630672.0,55.0,55.0,55.0,55.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaT,0
1,1,2015-01-02,S1F01085,0,1650864,56,0,52,6,407438,0,0,7,1.0,1,1650864.0,1650864.0,1650864.0,1650864.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaT,0
2,2,2015-01-03,S1F01085,0,124017368,56,0,52,6,407438,0,0,7,2.0,1,124017368.0,124017368.0,124017368.0,124017368.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407438.0,407438.0,407438.0,407438.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaT,0
3,3,2015-01-04,S1F01085,0,128073224,56,0,52,6,407439,0,0,7,3.0,1,128073224.0,128073224.0,128073224.0,128073224.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,407439.0,407439.0,407439.0,407439.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaT,0
4,4,2015-01-05,S1F01085,0,97393448,56,0,52,6,408114,0,0,7,4.0,1,97393448.0,97393448.0,97393448.0,97393448.0,56.0,56.0,56.0,56.0,0.0,0.0,0.0,0.0,52.0,52.0,52.0,52.0,6.0,6.0,6.0,6.0,408114.0,408114.0,408114.0,408114.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaT,0


In [59]:
#training_features['date'] = pd.to_numeric(pd.to_datetime(training_features['date']))
training_features[:5]

array([[0.00000000e+00, 8.83223757e-01, 8.46570619e-04, 0.00000000e+00,
        3.12124850e-02, 5.31914894e-02, 5.91203985e-01, 0.00000000e+00,
        0.00000000e+00, 3.74311534e-04, 8.83230558e-01, 8.83230558e-01,
        8.83223757e-01, 8.83230558e-01, 8.48975056e-04, 8.48975056e-04,
        8.46570619e-04, 8.48975056e-04, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 4.84171322e-02, 4.84171322e-02,
        3.12124850e-02, 4.84171322e-02, 5.31914894e-02, 5.31914894e-02,
        5.31914894e-02, 5.31914894e-02, 5.91203985e-01, 5.91203985e-01,
        5.91203985e-01, 5.91203985e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 3.74311534e-04, 3.74311534e-04,
        3.74311534e-04, 3.74311534e-04, 2.05077762e-01, 4.76190476e-02,
        0.00000000e+00, 4.76190476e-02, 4.06164892e-01, 3.04181048e-02,
        0.00000000e+00, 0.00000000e+00, 4.76190476e-02],
       

In [60]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC

#smx = SMOTENC(random_state=12,  categorical_features=[2])
smx = SMOTE(random_state=12)
x_res, y_res = smx.fit_sample(training_features.values, training_target.values.ravel())

AttributeError: ignored

In [ ]:
y_res[:5]

In [ ]:
df_x=pd.DataFrame(x_res)
df_x.columns = [
 'failure', 'metric1', 'metric2', 'metric3',
       'metric4', 'metric5', 'metric6', 'metric7', 'metric8', 'metric9',
       'metric1_mean', 'metric1_median',
       'metric1_max', 'metric1_min', 'metric2_mean', 'metric2_median',
       'metric2_max', 'metric2_min', 'metric3_mean', 'metric3_median',
       'metric3_max', 'metric3_min', 'metric4_mean', 'metric4_median',
       'metric4_max', 'metric4_min', 'metric5_mean', 'metric5_median',
       'metric5_max', 'metric5_min', 'metric6_mean', 'metric6_median',
       'metric6_max', 'metric6_min', 'metric7_mean', 'metric7_median',
       'metric7_max', 'metric7_min', 'metric8_mean', 'metric8_median',
       'metric8_max', 'metric8_min', 'metric9_mean', 'metric9_median',
       'metric9_max', 'metric9_min', 'metric1_chg', 'metric2_chg',
       'metric3_chg', 'metric4_chg', 'metric5_chg', 'metric6_chg',
       'metric7_chg', 'metric8_chg', 'metric9_chg']

In [ ]:
df_y=pd.DataFrame(y_res)
df_y.columns = ['FAILURE_TARGET']

In [ ]:
df_y.mean(axis = 0)

In [ ]:
df_balanced = pd.concat([df_y, df_x], axis=1)
print(df_balanced.shape)
df_balanced.head()

In [ ]:
df_balanced=df_balanced.drop(columns=['index'])

In [ ]:
features = [x for x in df_balanced.columns if x not in ['FAILURE_TARGET','EQUIPMENT_FAILURE']]  
dependent=pd.DataFrame(df_balanced['FAILURE_TARGET'])
independent=df_balanced.drop(columns=['FAILURE_TARGET'])
#make sure everything is numeric for simplicity
independent = independent.apply(pd.to_numeric) 
df_balanced = df_balanced.apply(pd.to_numeric)

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline
def evaluate_model(alg, train, target, predictors,  early_stopping_rounds=1):
    
   
    #Fit the algorithm on the data
    alg.fit(train[predictors], target['FAILURE_TARGET'], eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(train[predictors])
    dtrain_predprob = alg.predict_proba(train[predictors])[:,1]
    
    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False) 
    feat_imp.plot(kind='bar', title='Feature Importance', color='g') 
    plt.ylabel('Feature Importance Score')
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(target['FAILURE_TARGET'].values, dtrain_predictions))
    print("AUC Score (Balanced): %f" % metrics.roc_auc_score(target['FAILURE_TARGET'], dtrain_predprob))

In [ ]:
from xgboost.sklearn import XGBClassifier
xgb0 = XGBClassifier(
 objective= 'binary:logistic')

In [ ]:
evaluate_model(xgb0, independent, dependent,features)

In [ ]:
df_testing=df_train_test[df_train_test['MODELING_GROUP'] == 'TESTING'].copy()
df_training=df_train_test[df_train_test['MODELING_GROUP'] != 'TESTING'].copy()

In [ ]:
df_training['P_FAIL']= xgb0.predict_proba(df_training[features])[:,1];
df_training['Y_FAIL'] = np.where(((df_training.P_FAIL <= .50)), 0, 1)
#Print model report:
print("Accuracy : %.4g" % metrics.accuracy_score(df_training['FAILURE_TARGET'].values, df_training['Y_FAIL']))
print("AUC Score (Train): %f" % metrics.roc_auc_score(df_training['FAILURE_TARGET'], df_training['P_FAIL']))

In [ ]:
df_training['P_FAIL']= xgb0.predict_proba(df_training[features])[:,1];
df_training['Y_FAIL'] = np.where(((df_training.P_FAIL <= .67)), 0, 1)
#Print model report:
print("Accuracy : %.4g" % metrics.accuracy_score(df_training['FAILURE_TARGET'].values, df_training['Y_FAIL']))
print("AUC Score (Train): %f" % metrics.roc_auc_score(df_training['FAILURE_TARGET'], df_training['P_FAIL']))

In [ ]:
df_testing['P_FAIL']= xgb0.predict_proba(df_testing[features])[:,1];
df_testing['Y_FAIL'] = np.where(((df_testing.P_FAIL <= .67)), 0, 1)
#Print model report:
print("Accuracy : %.4g" % metrics.accuracy_score(df_testing['FAILURE_TARGET'].values, df_testing['Y_FAIL']))
print("AUC Score (Test): %f" % metrics.roc_auc_score(df_testing['FAILURE_TARGET'], df_testing['P_FAIL']))

In [ ]:
xxxx = pd.DataFrame(pd_data.groupby(['failure'])['device'].agg('count'))
xxxx

In [ ]:
print(pd.crosstab(df_testing.Y_FAIL, df_testing.failure, dropna=False))

In [ ]:
print(pd.crosstab(df_testing.Y_FAIL, df_testing.FAILURE_TARGET, dropna=False))

In [ ]:
forecast_window=90

In [ ]:
df_train_test['P_FAIL']= xgb0.predict_proba(df_train_test[features])[:,1];
df_train_test['Y_FAIL'] = np.where(((df_train_test.P_FAIL <= .67)), 0, 1)

In [ ]:
#sort the data by id and date.
xx=df_train_test
xx=xx.sort_values(by=['device','date'], ascending=[True, True])

In [ ]:
#create a unique list of machines
aa=xx

pd_id=aa.drop_duplicates(subset='device')
pd_id=pd_id[['device']]
pd_id.shape

In [ ]:
#label each machine with a sequential number
pd_id=pd_id.reset_index(drop=True)
pd_id=pd_id.reset_index(drop=False)
pd_id=pd_id.rename(columns={"index": "SCOOBYDOO"})
pd_id['SCOOBYDOO']=pd_id['SCOOBYDOO']+1
pd_id.head()

In [ ]:
#grab the max number of machines +1

column = pd_id["SCOOBYDOO"]
max_value = column.max()+1
max_value

In [ ]:
data_df['device'].nunique()
len(data_df.device.unique())

In [ ]:
#append sequential number to main file
xx=xx.sort_values(by=['device'], ascending=[True])
pd_id=pd_id.sort_values(by=['device'], ascending=[True])
xx =xx.merge(pd_id, on=['device'], how='inner')
xx.head()

In [ ]:
#sort data
xx=xx.sort_values(by=['device','date'], ascending=[True,True])

#reset index
xx=xx.reset_index(drop=True)

In [ ]:
#create a null dataframe for the next step
df_fred=xx
df_fred['Y_FAIL_sumxx']=0
df_fred=df_fred[df_fred['SCOOBYDOO'] == max_value+1]
df_fred.shape

In [ ]:
#sum the number of signals occuring over the last 90 days for each machine individually

for x in range(max_value):
        dffx=xx[xx['SCOOBYDOO'] ==x]
        dff=dffx.copy()
        dff['Y_FAIL_sumxx'] =(dff['Y_FAIL'].rolling(min_periods=1, window=(forecast_window)).sum())
        df_fred= pd.concat([df_fred,dff])

In [ ]:
xx=df_fred
xx.head(5)

In [ ]:
# if a signal has occured in the last 90 days, the signal is 0.
xx['Y_FAILZ']=np.where((xx.Y_FAIL_sumxx>1), 0, xx.Y_FAIL)

In [ ]:
#sort the data by id and date.

xx=xx.sort_values(by=['device','date'], ascending=[True, True])

In [ ]:
#create signal id with the cumsum function.
xx['SIGNAL_ID'] = xx['Y_FAILZ'].cumsum()

In [ ]:
df_signals=xx[xx['Y_FAILZ'] == 1]
df_signal_date=df_signals[['SIGNAL_ID','date','device']]
df_signal_date=df_signal_date.rename(index=str, columns={"date": "SIGNAL_DATE"})
df_signal_date=df_signal_date.rename(index=str, columns={"device": "ID_OF_SIGNAL"})

In [ ]:
df_signal_date.shape

In [ ]:
xx =xx.merge(df_signal_date, on=['SIGNAL_ID'], how='outer')